In [72]:
import pandas as pd 
import numpy as np
import sqlalchemy
import pyodbc

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.window as w
import pyspark.sql.types as t
import pyspark.pandas as ps

In [15]:
spark = SparkSession.builder.appName('TheVoice-ETL').getOrCreate()

In [16]:
# Load into dataframes
call_type_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\call_type.parquet')
countries_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\countries.parquet')
customer_mrr =          spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer.parquet')
customer_invoice_mrr =  spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer_invoice.parquet')
customer_lines_mrr =    spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer_lines.parquet')
opfileopp_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\pfileopp.parquet')
package_catalog_mrr =   spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\package_catalog.parquet')
usage_main_mrr =        spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\usage_main.parquet')
xxCountryType_mrr =     spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\xxCountryType.parquet')

In [17]:
# DimCallTypes staging

# variable set
price_per_minute = 0.5

# window for the key column
windowSpec_callType = w.Window.orderBy('call_type_code')

# final df 
DimCallTypes_stg = call_type_mrr.withColumn('KeyCallType', f.row_number().over(windowSpec_callType) + 1000)\
    .withColumnRenamed('call_type_code', 'DescCallTypeCode')\
    .withColumnRenamed('call_type_desc', 'DescCallType')\
    .withColumn('DescFullCallType', 
        f.concat_ws('-', f.col('DescCallTypeCode'), f.col('DescCallType')))\
    .withColumn('DescCallTypePriceCategory', f.when(f.col('priceperminuter') > price_per_minute, 'Discounted Price').otherwise('Normal Price'))\
    .withColumnRenamed('call_type', 'DescCallTypeCategory')\
    .select(
        'KeyCallType', 
        'DescCallTypeCode', 
        'DescCallType', 
        'DescFullCallType', 
        'DescCallTypePriceCategory', 
        'DescCallTypeCategory'
    )#.show()


In [18]:
# DimCountries staging

# select proper column and rename the country code column
xxCountryType_mrr = xxCountryType_mrr.select('COUNTRY_CODE', 'COUNTRY_PRE').withColumnRenamed('COUNTRY_CODE','COUNTRY_CODE2')

# final df
DimCountries_stg = countries_mrr.select('COUNTRY_CODE', 'REGION', 'AREA').join(
    xxCountryType_mrr,
    countries_mrr['COUNTRY_CODE'] == xxCountryType_mrr['COUNTRY_CODE2'],
    how='inner')\
    .withColumnRenamed('COUNTRY_PRE', 'KeyCountry')\
    .withColumnRenamed('COUNTRY_CODE', 'DescCountry')\
    .withColumnRenamed('REGION', 'DescRegion')\
    .withColumnRenamed('AREA', 'DescArea')\
    .select('KeyCountry', 'DescCountry', 'DescRegion', 'DescArea')#.show(5)

In [19]:
# DimPackageCatalog staging
DimPackageCatalog_stg = package_catalog_mrr.withColumnRenamed('PACKAGE_NUM','KeyPackage')\
.withColumnRenamed('pack_desc', 'DescPackage')\
.withColumnRenamed('createdate', 'DatePackageCreation')\
.withColumnRenamed('enddate', 'DatePackageEnd')\
.withColumn('DescPackageStatus',
    f.when(f.col('status') == 1, 'Active').otherwise('Inactive'))\
.withColumn('CodePackageActivitiesDays', f.date_diff(f.col('DatePackageEnd'), f.col('DatePackageCreation')))

In [143]:
DimPackageCatalog_stg.show(5)

+----------+--------------------+--------------+------+---------+--------------------+--------------------+--------------------+-----------------+-------------------------+
|KeyPackage| DatePackageCreation|DatePackageEnd|status|pack_type|         DescPackage|         insert_date|         update_date|DescPackageStatus|CodePackageActivitiesDays|
+----------+--------------------+--------------+------+---------+--------------------+--------------------+--------------------+-----------------+-------------------------+
|         1|2014-01-15 16:28:...|          NULL|     1|   FAMILY|pay much more get...|2014-01-15 16:28:...|2014-01-15 16:28:...|           Active|                     NULL|
|         2|2014-01-15 16:28:...|          NULL|     1|  FRIENDS|friends pay more ...|2014-01-15 16:28:...|2014-01-15 16:28:...|           Active|                     NULL|
|         3|2014-01-15 16:28:...|          NULL|     1| BUSINESS|business man pay ...|2014-01-15 16:28:...|2014-01-15 16:28:...|       

In [20]:
opfileopp_mrr.show(n=5)

+-----+----------+------+
|OPCCC|     OPDDD|prepre|
+-----+----------+------+
|   50|    פלאפון|    50|
|   52|     סלקום|    52|
|   53|הוט מובייל|    53|
|   54|     אורנג|    54|
|   58|גולן טלקום|    58|
+-----+----------+------+



In [21]:
# DimOperators staging
DimOperators_stg = opfileopp_mrr.withColumnRenamed('OPCCC', 'KeyOperator')\
    .withColumn('DescOperator', f.concat_ws('-', f.col('prepre').cast('string'), f.col('OPDDD').cast('string')))\
    .withColumnRenamed('prepre', 'DescKeyPrefix')\
    .select('KeyOperator', 'DescOperator', 'DescKeyPrefix')

In [57]:
# DimCustomers staging
customer_mrr = spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer.parquet')
DimCustomers_stg = customer_mrr.join(customer_lines_mrr.select('PHONE_NO', 'DESC'), 
        customer_mrr['CUST_NUMBER'] == customer_lines_mrr['PHONE_NO'], how='inner')\
    .withColumnRenamed('customer_id', 'KeyCustomer')\
    .withColumnRenamed('DESC', 'DescCusomterPackage')\
    .withColumn( 'CustomerLineOperatorPrefixForJoin',
        f.when(f.length(f.col('PHONE_NO').cast('string')) == 12, 
            f.substr(f.col('PHONE_NO').cast('string'), f.lit(4), f.lit(2)))\
            .otherwise(f.substr(f.col('PHONE_NO').cast('string'), f.lit(2), f.lit(3)))
    )\
    .withColumn('CustomercountryPrefixForJoin',
        f.when(f.length(f.col('PHONE_NO').cast('string')) == 12, 
            f.substr(f.col('PHONE_NO').cast('string'), f.lit(1), f.lit(3)))\
            .otherwise(f.substr(f.col('PHONE_NO').cast('string'), f.lit(1), f.lit(1)))
    )\
    .withColumnRenamed('cust_name', 'DescCustomerName')\
    .withColumnRenamed('address', 'DescCustomerAddress')\
    .join(opfileopp_mrr.select('OPCCC', 'OPDDD'),#
        f.col('CustomerLineOperatorPrefixForJoin').cast('int') == f.col('OPCCC').cast('int'), how='left')\
    .withColumn('operatorName',
         f.when(f.isnull(f.col('OPDDD')), 'Unknown').otherwise(f.col('OPDDD')))\
    .withColumnRenamed('operatorName', 'DescCustomerLineOperator')\
    .join(xxCountryType_mrr,
        f.col('CustomerCountryPrefixForJoin') == f.col('COUNTRY_PRE'), how='left'
    )\
    .withColumn('DescCustomerLineCountry',
        f.when(f.isnull('COUNTRY_CODE2'), 'Unknown').otherwise(f.col('COUNTRY_CODE2'))
    )\
    .select(
        'KeyCustomer', 
        'DescCustomerLineOperator', 
        'DescCustomerLineCountry', 
        'DescCustomerName', 
        'DescCustomerAddress', 
        'DescCusomterPackage'
    )
    #.orderBy(f.col('CustomerLineOperatorPrefixForJoin').asc())

In [63]:
# DimCallOriginType staging
DimCallOriginType_stg = usage_main_mrr.withColumn('DescCallOriginType',
    f.when(f.col('CELL_ORIGIN') == 1, 'Cellular Call')\
    .when(f.col('CELL_ORIGIN') == 0 , 'Cellular Call').otherwise('Unknown')
).withColumnRenamed('CELL_ORIGIN', 'KeyCallOriginType')\
.select('KeyCallOriginType', 'DescCallOriginType')

In [107]:
# DimDate staging
start_date = usage_main_mrr.select(f.min(f.col('CALL_DATETIME'))).first()[0]
end_date = usage_main_mrr.select(f.max(f.col('CALL_DATETIME'))).first()[0]

DimDate_pandas = pd.DataFrame({'FullDate': pd.date_range(start= start_date, end= end_date)})
DimDate_stg = spark.createDataFrame(DimDate_pandas)

DimDate_stg = DimDate_stg.withColumn('month_string',
    f.when(f.month(f.col('FullDate')) < 10, f.concat(f.lit('0'), f.month(f.col('FullDate'))))\
    .otherwise(f.month(f.col('FullDate')))
)\
.withColumn('day_string',
    f.when(f.day(f.col('FullDate')) < 10, f.concat(f.lit('0'), f.day(f.col('FullDate'))))\
    .otherwise(f.day(f.col('FullDate')))
)\
.withColumn('KeyDate', f.concat(f.year(f.col('FullDate')), f.col('month_string'), f.col('day_string')))\
.withColumn('KeyYear',f.year(f.col('FullDate')))\
.withColumn('KeyMonth', f.concat(f.year('FullDate'), f.col('month_string')))\
.withColumn('CodeMonth', f.month(f.col('FullDate')))\
.withColumn('DescMonth', f.date_format(f.col('FullDate'), 'MMMM'))\
.withColumn('CodeDayInWeek', f.day(f.col('FullDate')))\
.withColumn('DescDayInWeek', f.date_format(f.col('FullDate'), 'EEEE'))\
.select(
    'KeyDate', 
    'FullDate', 
    'KeyYear', 
    'KeyMonth', 
    'CodeMonth', 
    'DescMonth', 
    'CodeDayInWeek', 
    'DescDayInWeek'
)#.show(5)

In [150]:
# FactUsage staging
usage_main_mrr.join(customer_mrr.select(f.col('customer_id')),
    f.col('CALL_NO') == f.col('customer_id'), how='inner')\
    .withColumnRenamed('CALL_NO', 'CallId')\
    .withColumnRenamed('CUST_ID', 'KeyCustomer')\
    .join(DimCallTypes_stg.select('KeyCallType', 'DescCallTypeCode'),
        f.col('CALL_TYPE') == f.col('DescCallTypeCode'), how='inner')\
    .withColumn('CALLING_NO_operator',
        f.when(f.length(f.col('CALLING_NO')) == 12, 
            f.substr(f.col('CALLING_NO'), f.lit(4), f.lit(2)))\
            .otherwise(f.substr(f.col('CALLING_NO'), f.lit(2), f.lit(3))))\
    .join(opfileopp_mrr.select('OPCCC', 'OPDDD'),
        f.col('CALLING_NO_operator') == f.col('OPCCC'), how='left')\
    .withColumn('KeyOriginOperator',
        f.when(f.isnull(f.col('OPDDD')), f.lit(-1)).otherwise(f.col('OPDDD')))\
    .withColumn('CALLING_NO_country',
        f.when(f.length(f.col('CALLING_NO')) == 12, 
            f.substr(f.col('CALLING_NO'), f.lit(1), f.lit(3)))\
            .otherwise(f.substr(f.col('CALLING_NO'), f.lit(1), f.lit(1))))\
    .join(xxCountryType_mrr,
        f.col('CALLING_NO_country') == f.col('COUNTRY_PRE'), how='left')\
    .withColumn('KeyOriginCountry',
        f.when(f.isnull(f.col('COUNTRY_CODE2')), f.lit(-1)).otherwise(f.col('COUNTRY_CODE2')))\
    .withColumn('DES_NO_operator',
        f.when(f.length(f.col('DES_NO')) == 12, 
            f.substr(f.col('DES_NO'), f.lit(4), f.lit(2)))\
            .otherwise(f.substr(f.col('DES_NO'), f.lit(2), f.lit(3))))\
    .join(opfileopp_mrr.select(f.col('OPCCC').alias('OPCCC_DES_NO'), f.col('OPDDD').alias('OPDDD_DES_NO')),
        f.col('DES_NO_operator') == f.col('OPCCC_DES_NO'), how='left')\
    .withColumn('KeyDestinationOperator',
        f.when(f.isnull(f.col('OPDDD_DES_NO')), f.lit(-1)).otherwise(f.col('OPDDD_DES_NO')))\
    .join(customer_lines_mrr,
        f.col('CALLING_NO') == f.col('PHONE_NO'), how='inner')\
    .join(DimPackageCatalog_stg,
        f.col('TYPE') == f.col('pack_type'), how='inner')\
    .withColumnRenamed('CELL_ORIGIN', 'KeyCallOriginType')\
    .withColumn('month_string',
        f.when(f.month(f.col('CALL_DATETIME')) < 10, f.concat(f.lit('0'), f.month(f.col('CALL_DATETIME'))))\
        .otherwise(f.month(f.col('CALL_DATETIME'))))\
    .withColumn('day_string',
        f.when(f.day(f.col('CALL_DATETIME')) < 10, f.concat(f.lit('0'), f.day(f.col('CALL_DATETIME'))))\
        .otherwise(f.day(f.col('CALL_DATETIME'))))\
    .withColumn('KeyCallDate', f.concat(f.year(f.col('CALL_DATETIME')), f.col('month_string'), f.col('day_string')))\
    .select(
        'CallId', 
        'KeyCustomer', 
        'KeyCallType', 
        'KeyOriginOperator', 
        'KeyOriginCountry', 
        'KeyDestinationOperator', 
        'KeyPackage',
        'KeyCallDate',
        ).show(5)


+------+-----------+-----------+-----------------+----------------+----------------------+----------+
|CallId|KeyCustomer|KeyCallType|KeyOriginOperator|KeyOriginCountry|KeyDestinationOperator|KeyPackage|
+------+-----------+-----------+-----------------+----------------+----------------------+----------+
|     1|       8544|       1036|            סלקום|          Israel|                    -1|         1|
|     2|      10205|       1033|               -1|   United States|                    -1|         1|
|     3|      18147|       1036|               -1|   United States|                    -1|         3|
|     4|       9327|       1033|            סלקום|          Israel|                 סלקום|         1|
|     5|      17731|       1002|               -1|   United States|                    -1|         1|
+------+-----------+-----------+-----------------+----------------+----------------------+----------+
only showing top 5 rows



In [144]:
DimPackageCatalog_stg.show(5)

+----------+--------------------+--------------+------+---------+--------------------+--------------------+--------------------+-----------------+-------------------------+
|KeyPackage| DatePackageCreation|DatePackageEnd|status|pack_type|         DescPackage|         insert_date|         update_date|DescPackageStatus|CodePackageActivitiesDays|
+----------+--------------------+--------------+------+---------+--------------------+--------------------+--------------------+-----------------+-------------------------+
|         1|2014-01-15 16:28:...|          NULL|     1|   FAMILY|pay much more get...|2014-01-15 16:28:...|2014-01-15 16:28:...|           Active|                     NULL|
|         2|2014-01-15 16:28:...|          NULL|     1|  FRIENDS|friends pay more ...|2014-01-15 16:28:...|2014-01-15 16:28:...|           Active|                     NULL|
|         3|2014-01-15 16:28:...|          NULL|     1| BUSINESS|business man pay ...|2014-01-15 16:28:...|2014-01-15 16:28:...|       

In [151]:
print(customer_mrr.columns)
usage_main_mrr.columns

['customer_id', 'CUST_NUMBER', 'cust_name', 'address', 'insert_date', 'update_date']


['CALL_NO',
 'ANSWER_TIME',
 'SEIZED_TIME',
 'DISCONNECT_TIME',
 'CALL_DATETIME',
 'CALLING_NO',
 'CALLED_NO',
 'DES_NO',
 'DURATION',
 'CUST_ID',
 'CALL_TYPE',
 'PROD_TYPE',
 'RATED_AMNT',
 'RATED_CURR_CODE',
 'CELL',
 'CELL_ORIGIN',
 'HIGH_LOW_RATE',
 'insert_DATE',
 'update_date']